In [1]:
from hero_db_utils.datamodels import datamodel

In [2]:
@datamodel
class Triangle:
    sidea:float
    sideb:float
    sidec:float
    
    def is_equilateral(self):
        return self.sidea == self.sideb == self.sidec

In [6]:
mytriangles = Triangle.asdf([Triangle(5, 5, "5"), Triangle(10,5,2), Triangle(2,2,2)])
mytriangles

,sidea,sideb,sidec
0,5.0,5.0,5.0
1,10.0,5.0,2.0
2,2.0,2.0,2.0
